In [20]:
from datasets import load_dataset

# load from parquet file (~4000 samples in a parquet file)
# link to other parquet files: https://huggingface.co/datasets/linhtran92/viet_bud500/tree/main/data

train_url = "https://huggingface.co/datasets/linhtran92/viet_bud500/resolve/main/data/train-00000-of-00105-be5f872f8be772f5.parquet"
test_url = "https://huggingface.co/datasets/linhtran92/viet_bud500/resolve/main/data/test-00000-of-00002-531c1d81edb57297.parquet"

data_files = {"train": train_url, "test" : test_url}
dataset = load_dataset("parquet", data_files=data_files, )

# # load dataset via streaming
# dataset = load_dataset("linhtran92/viet_bud500", split='test', streaming=True)
# dataset.take(2)

# # load all (649158 samples, ~100gb, ~2hrs to complete loading)
# dataset = load_dataset("linhtran92/viet_bud500", split="test")

In [21]:
import soundfile as sf

In [22]:
def convert_waveform_to_audio(waveform_list, sampling_rate=24000, path_to_audio='',prefix='train'):
    count = 0
    for waveform in waveform_list:
        print(waveform)
        sf.write(f'{path_to_audio}{prefix}_{str(count).zfill(10)}.wav', waveform['array'], sampling_rate)
        count += 1
    print('saved all audio files!') 
sampling_rate = 24000
# for audio in dataset['train']['audio']:
#     sf.write(f'../dataset/audio/train_24k/train24k_{str(count).zfill(10)}.wav', audio['array'], sampling_rate)
#     count += 1
# count = 0
# for audio in dataset['test']['audio']:
#     sf.write(f'../dataset/audio/test_24k/test24k_{str(count).zfill(10)}.wav', audio['array'], sampling_rate)
#     count += 1

In [24]:
%ls

 Volume in drive D has no label.
 Volume Serial Number is 6F86-43F0

 Directory of d:\AI_CODE\BunStyleTTS2

09/05/2024  02:40 PM    <DIR>          .
09/08/2024  03:07 PM    <DIR>          ..
09/11/2024  09:26 PM    <DIR>          dataset
09/05/2024  02:41 PM               126 fix.txt
09/11/2024  07:47 PM    <DIR>          notebook
09/04/2024  10:52 PM            19,344 setup.ipynb
09/04/2024  10:55 PM    <DIR>          StyleTTS2
09/04/2024  10:35 PM    <DIR>          StyleTTS2-LibriTTS
               2 File(s)         19,470 bytes
               6 Dir(s)  79,760,912,384 bytes free


In [27]:
# convert_waveform_to_audio(dataset['train']['audio'], sampling_rate=24000,path_to_audio='../dataset/audio/train_24k/',prefix='train24k')
convert_waveform_to_audio(dataset['test']['audio'], sampling_rate=24000,path_to_audio='dataset/audio/test_24k/',prefix='test24k')

{'path': None, 'array': array([0.02420044, 0.04144287, 0.03671265, ..., 0.0375061 , 0.04605103,
       0.05227661]), 'sampling_rate': 16000}
{'path': None, 'array': array([-0.00128174, -0.00125122, -0.00167847, ..., -0.17169189,
       -0.15856934, -0.14556885]), 'sampling_rate': 16000}
{'path': None, 'array': array([-0.00796509, -0.01101685, -0.01052856, ...,  0.20025635,
        0.04891968, -0.06576538]), 'sampling_rate': 16000}
{'path': None, 'array': array([0.00674438, 0.01101685, 0.00924683, ..., 0.07568359, 0.08120728,
       0.08947754]), 'sampling_rate': 16000}
{'path': None, 'array': array([-0.28063965, -0.43130493, -0.28204346, ..., -0.06674194,
       -0.01989746,  0.0149231 ]), 'sampling_rate': 16000}
{'path': None, 'array': array([-0.1104126 , -0.20974731, -0.20562744, ..., -0.19830322,
       -0.2182312 , -0.24847412]), 'sampling_rate': 16000}
{'path': None, 'array': array([0.07781982, 0.11328125, 0.09005737, ..., 0.02062988, 0.02368164,
       0.02484131]), 'sampling_rat

In [4]:
import phonemizer
global_phonemizer = phonemizer.backend.EspeakBackend(language='vi', preserve_punctuation=True,  with_stress=True)

In [6]:
from phonemizer import phonemize
text = "Xin chào, tôi là một lập trình viên."

# Sử dụng phonemizer để chuyển đổi văn bản thành IPA
ipa_text = phonemize(text, language='vi', backend='espeak', strip=True, preserve_punctuation=True)

print(ipa_text)

sin tʃaː2w, t̪oj laː2 mo6t̪ lə6p tʃi2ɲ viɛn.


In [5]:
def create_data_list(transcription_list, path_to_audio, path_to_list, prefix, language):
    count = 0
    global_phonemizer = phonemizer.backend.EspeakBackend(language=language, preserve_punctuation=True,  with_stress=True)
    with open(path_to_list, 'w') as f:
        for text in transcription_list:
            text = text.strip()
            ps = global_phonemizer.phonemize([text])
            f.write(f'{path_to_audio}/{prefix}_{str(count).zfill(10)}.wav|{ps[0]}|0\n')
            count += 1

# with open('../dataset/train_list_24k.txt', 'w') as f:
#     count = 0
#     for text in dataset['train']['transcription']:
#         text = text.strip()
#         ps = global_phonemizer.phonemize([text])
#         f.write(f'train_24k/train24k_{str(count).zfill(10)}.wav|{ps[0]}|0\n')
#         count += 1
# with open('../dataset/test_list_24k.txt', 'w') as f:
#     count = 0
#     for text in dataset['test']['transcription']:
#         text = text.strip()
#         ps = global_phonemizer.phonemize([text])
#         f.write(f'test_24k/test24k_{str(count).zfill(10)}.wav|{ps[0]}|0\n')
#         count += 1

In [16]:
create_data_list(dataset['train']['transcription'],'train_24k','dataset/train_list_24k_new.txt','train24k','vi')

In [17]:
create_data_list(dataset['test']['transcription'],'test_24k','dataset/test_list_24k_new.txt','test24k','vi')